# Deep Learning Diagnose on Brain MRI Images using PyTorch

## Create Dataset and Dataloader

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
import numpy as np

classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

class TumorDataset(Dataset):
    def __init__(self, annotations_file):
        self.imgs_info = pd.read_csv(annotations_file, header=None)

    def __len__(self):
        return len(self.imgs_info)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_path = self.imgs_info.iloc[idx, 0]
        img_raw = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        img_resize = cv.resize(img_raw, (128, 128))
        image = np.float32(img_resize / 255)
        if self.imgs_info.iloc[idx, 1] == classes[0]:
            category = 0
        elif self.imgs_info.iloc[idx, 1] == classes[1]:
            category = 1
        elif self.imgs_info.iloc[idx, 1] == classes[2]:
            category = 2
        else:
            category = 3
        sample = {'image': image, 'category': category}
        return sample
    
dataset_train = TumorDataset(annotations_file='annotation_train.csv')
# for i, sample in enumerate(dataset_train):
#     image = sample['image']
#     label = sample['category']
#     if not i%100:  # i % 100 != 0
#         print(i, image.shape, label)
# print(i, image.shape, label)
dataset_test = TumorDataset(annotations_file='annotation_test.csv')

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True)

for i, sample_batch in enumerate(dataloader_train):
    print(f"Shape of X [N, H, W]: {sample_batch['image'].shape}")
    print(f"Shape of y: {sample_batch['category'].shape}")
    break

Shape of X [N, H, W]: torch.Size([256, 128, 128])
Shape of y: torch.Size([256])


## Create a MLP Model

In [29]:
from torch import nn

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*128, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

mlp = MultiLayerPerceptron().to(device)
print(mlp)

Using cuda device
MultiLayerPerceptron(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16384, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=4, bias=True)
  )
)


## Training and Testing on One Batch

In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for i, batch in enumerate(dataloader):
        X, y = batch['image'].to(device), batch['category'].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
            loss, current = loss.item(), (i + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch['image'].to(device), batch['category'].to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Optimize Model


In [ ]:
mlp = MultiLayerPerceptron().to(device)  # initialize params
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr=1e-3)
epochs = 15

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader_train, mlp, loss_fn, optimizer)
    test(dataloader_test, mlp, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 1.357307  [  256/ 5712]
loss: 1.351721  [ 2816/ 5712]
loss: 1.356101  [ 5376/ 5712]
Test Error: 
 Accuracy: 41.0%, Avg loss: 1.352920 

Epoch 2
-------------------------------
loss: 1.364135  [  256/ 5712]


KeyboardInterrupt: 